In [10]:
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    

def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(url,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url



In [14]:
search = input("검색할 키워드를 입력해주세요:")
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")

page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")





크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지:  3 페이지


In [19]:
# URL 생성
url_list = makeUrl(search, page, page2)

# 결과 저장 리스트 초기화
news_titles = []
news_url_nested = []  # 중첩 리스트
news_contents = []

# 뉴스 URL 수집
for single_url in url_list:
    result = articles_crawler(single_url)
    news_url_nested.append(result)

# 중첩 리스트 평탄화 함수 정의
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

# 평탄화 실행
news_url_flat = []
makeList(news_url_flat, news_url_nested)





생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=물류&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=물류&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=물류&start=21']


[]

In [21]:
url_list

['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=물류&start=1',
 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=물류&start=11',
 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=물류&start=21']

In [20]:
news_url_flat

[]

In [ ]:
# 네이버 뉴스 URL만 필터링
final_urls = []
for i in tqdm(range(len(news_url_flat))):
    if "news.naver.com" in news_url_flat[i]:
        final_urls.append(news_url_flat[i])
    else:
        pass


In [ ]:

# 검색할 키워드를 입력해주세요:물류
# 크롤링할 시작 페이지를 입력해주세요.
# 크롤링할 시작 페이지: 1 페이지
# 크롤링할 종료 페이지를 입력해주세요.
# 크롤링할 종료 페이지: 3 페이지

In [17]:
for i in tqdm(final_urls):
    
    news = requests.get(i, headers=headers)
    news_html = BeautifulSoup(news.text, "html.parser")

    # 제목 크롤링
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title is None:
        title = news_html.select_one("#content > div.end_ct > div > h2")

    # 본문 크롤링
    content = news_html.select("article#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 태그 제거 및 전처리
    content = ''.join(str(content))
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)

    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    # 리스트에 저장
    news_titles.append(title)
    news_contents.append(content)


0it [00:00, ?it/s]


In [18]:
print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)
print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))

검색된 기사 갯수: 총  30 개

[뉴스 제목]
[]

[뉴스 링크]
[]

[뉴스 내용]
[]
news_title:  0
news_url:  0
news_contents:  0
